## 1. CÀI ĐẶT THƯ VIỆN 

In [ ]:

!pip install -q ultralytics albumentations opencv-python

import os
import cv2
import albumentations as A
from glob import glob
from tqdm import tqdm
from ultralytics import YOLO
import shutil


## 2. ĐƯỜNG DẪN

In [ ]:

DATASET_DIR = "/kaggle/input/vietnamese-traffic-signs-detection-and-recognition/train_data"  # 🟡 Thay đổi tên thư mục nếu cần
WORK_DIR = "/kaggle/working/dataset"

# Sao chép dữ liệu sang working directory (để có quyền ghi)
!cp -r {DATASET_DIR} {WORK_DIR}

# ==== Đường dẫn dataset ====
image_dir = os.path.join(WORK_DIR, "images/train")
label_dir = os.path.join(WORK_DIR, "labels/train")

aug_image_dir = os.path.join(WORK_DIR, "images/train_aug")
aug_label_dir = os.path.join(WORK_DIR, "labels/train_aug")
os.makedirs(aug_image_dir, exist_ok=True)
os.makedirs(aug_label_dir, exist_ok=True)

## 3. CẤU HÌNH AUGMENTATION

In [ ]:
# ==== Số lượng ảnh tăng cường cho mỗi ảnh gốc ====
AUG_PER_IMAGE = 10

# ==== Kích thước resize đích (YOLO phổ biến dùng 416x416) ====
TARGET_SIZE = 416

# ==== Cấu hình augmentations ====
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.MotionBlur(p=0.3),
    A.Rotate(limit=15, p=0.5),
    A.Affine(scale=(0.8, 1.2), translate_percent=(0.1, 0.1), p=0.5),
    A.Resize(height=TARGET_SIZE, width=TARGET_SIZE)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

# ==== Thực hiện augment ====
image_paths = glob(os.path.join(image_dir, "*.png"))

for img_path in tqdm(image_paths):
    filename = os.path.basename(img_path)
    label_path = img_path.replace("images", "labels").replace(".png", ".txt")

    image = cv2.imread(img_path)
    if image is None or not os.path.exists(label_path):
        continue

    with open(label_path, "r") as f:
        lines = f.readlines()

    bboxes = []
    class_labels = []

    for line in lines:
        parts = line.strip().split()
        cls = int(parts[0])
        bbox = list(map(float, parts[1:]))
        bboxes.append(bbox)
        class_labels.append(cls)

    for i in range(AUG_PER_IMAGE):
        augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_classes = augmented['class_labels']

        # Lưu dưới dạng .jpg
        aug_filename = filename.replace(".png", f"_aug{i}.jpg")
        aug_labelname = filename.replace(".png", f"_aug{i}.txt")

        cv2.imwrite(
            os.path.join(aug_image_dir, aug_filename),
            aug_img,
            [cv2.IMWRITE_JPEG_QUALITY, 90]  # Giảm dung lượng ảnh
        )

        with open(os.path.join(aug_label_dir, aug_labelname), "w") as f:
            for cls, bbox in zip(aug_classes, aug_bboxes):
                f.write(f"{cls} {' '.join(map(str, bbox))}\n")

# ==== Gộp dữ liệu gốc và dữ liệu augment ====
!cp {aug_image_dir}/* {image_dir}/
!cp {aug_label_dir}/* {label_dir}/


## 4. GỘP DỮ LIỆU GỐC VÀ DỮ LIỆU TĂNG CƯỜNG

In [ ]:

# Gộp ảnh và nhãn tăng cường vào thư mục train gốc
!cp {aug_image_dir}/* {image_dir}/
!cp {aug_label_dir}/* {label_dir}/

data_yaml = f"""
path: {WORK_DIR}
train: images/train
val: images/val
test: images/test

nc: 29
names:
  0: one way prohibition
  1: no parking
  2: no stopping and parking
  3: no turn left
  4: no turn right
  5: no u turn
  6: no u and left turn
  7: no u and right turn
  8: no motorbike entry/turning
  9: no car entry/turning
  10: no truck entry/turning
  11: other prohibition
  12: indication
  13: direction
  14: speed limit
  15: weight limit
  16: height limit
  17: pedestrian crossing
  18: intersection danger
  19: road danger
  20: pedestrian danger
  21: construction danger
  22: slow warning
  23: other warning
  24: vehicle permission lane
  25: vehicle and speed permission lane
  26: overpass route
  27: no more prohibition
  28: other
"""

with open(os.path.join(WORK_DIR, "data.yaml"), "w") as f:
    f.write(data_yaml)


## 5. TẢI DATASET

In [ ]:
!zip -r /kaggle/working/dataset.zip /kaggle/working/dataset


In [ ]:
!ls -R /kaggle/working/dataset


In [ ]:


# Tạo thư mục outputs nếu chưa tồn tại
os.makedirs("/kaggle/outputs", exist_ok=True)

# Copy file zip sang outputs
shutil.copy("/kaggle/working/dataset.zip", "/kaggle/outputs/dataset.zip")
